In [1]:
!pip install tensorflow==1.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.6/367.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.7 MB/s eta 0:00:0000:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.4
    Uninstalling tensorflow-2.6.4:
      Successfully uninstalled tensorflow-2.6.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [2]:
import tensorflow as tf
import os
import time
import pickle
import numpy as np

print(tf.__version__)
CIFAR_DIR = '/kaggle/input/cifar10/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1
['data_batch_1', 'data_batch_2', 'batches.meta', 'test_batch', 'data_batch_3', 'data_batch_5', 'data_batch_4', 'readme.html']


In [3]:
train_filenames = [os.path.join(CIFAR_DIR, f'data_batch_{i}') for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]
print(train_filenames)
print(test_filenames)

['/kaggle/input/cifar10/cifar-10-batches-py/data_batch_1', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_2', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_3', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_4', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_5']
['/kaggle/input/cifar10/cifar-10-batches-py/test_batch']


In [4]:
print(np.random.permutation(10))

[1 2 4 7 9 0 8 6 5 3]


In [5]:
def load_data(filename):
    """前面是图片特征，后面是图片分类标签"""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']


# tensorflow.Dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_lables = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)  # all_data[0]列表中10000个存有(32*32*3=)3072个数的一维列表c
            all_lables.append(labels)  # all_data[0]列表中10000个标签值
        self._data = np.vstack(all_data)  # 垂直方向沿轴2堆叠 变为ndarray
        self._data = self._data / 127.5 - 1  # 归一化到范围[-1,1]
        self._lables = np.hstack(all_lables)  # 一维列表变成ndarray
        print(self._data.shape)  # (50000, 3072) 每一张图像展平后的结果
        print(self._lables.shape)
        print(self._lables[:5])

        self._num_samples = self._data.shape[0]  # 样本数
        self._need_shuffle = need_shuffle  # 进行洗牌随机
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()

    def _shuffle_data(self):
        # 通过随机排列下标进行随机样本
        p = np.random.permutation(self._num_samples)  # 变成随机下标
        self._data = self._data[p]
        self._lables = self._lables[p]

    def next_batch(self, batch_size):
        """return batch_size samples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_samples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = self._indicator + batch_size
            else:
                raise Exception('have no more samples')
        if end_indicator > self._num_samples:
            # 去除不足batch_size剩余的样本
            raise Exception('batch size is larger than the remaining samples')
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._lables[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels


train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
[6 9 9 4 1]
(10000, 3072)
(10000,)
[3 8 8 0 6]


In [6]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
x_image = tf.reshape(x, [-1, 3, 32, 32])  # 原数据channels_first存储的
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])  # 功类似np.rollaxis轴滚动

In [7]:

conv1_1 = tf.layers.conv2d(x_image,
                           32,  # output channel number
                           (3, 3),  # kernel size
                           # 明确指定stride时padding='same'也不能保持形状不变 就像output_size等于input_size/stride
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv1_1')
conv1_2 = tf.layers.conv2d(conv1_1,
                           32,
                           (3, 3),
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv1_2')

pooling1 = tf.layers.max_pooling2d(conv1_2,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name='pool1')
# pooling1输出形状 16 * 16 * 32
conv2_1 = tf.layers.conv2d(pooling1,
                           32,
                           (3, 3),
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv2_1')
conv2_2 = tf.layers.conv2d(conv2_1,
                           32,
                           (3, 3),
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv2_2')
pooling2 = tf.layers.max_pooling2d(conv2_2,
                                   (2, 2),
                                   (2, 2),
                                   name='pool2')
# pooling2输出形状 8 * 8 * 32
conv3_1 = tf.layers.conv2d(pooling2,
                           32,
                           (3, 3),
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv3_1')
conv3_2 = tf.layers.conv2d(conv3_1,
                           32,
                           (3, 3),
                           padding='same',
                           activation=tf.nn.relu,
                           name='conv3_2')
# # pooling3输出形状 4 * 4 * 32
pooling3 = tf.layers.max_pooling2d(conv3_2,
                                   (2, 2),  # kernel size
                                   (2, 2),  # stride
                                   name='pool3')
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)
# 深度学习中，logits就是最终的全连接层的输出
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# softmax(y_)->p->one_hot->loss=plog(p)
predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)  # y是真实值
# 计算张量维度上元素的平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
with tf.name_scope('train_op'):
    # 这里计算梯度并更新了梯度
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [10]:
init = tf.global_variables_initializer()  # 低版本tf必须要做
batch_size = 20
train_steps = 10000
test_steps = 100
with tf.Session() as s:
    s.run(init)  # 低版本tf必须要做
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = s.run([loss, accuracy, train_op],
                                      feed_dict={x: batch_data, y: batch_labels})
        if (i+1)%100==0:
            print('[Train] Step: %d, loss: %4.5f, accuracy: %4.5f'
                  % (i+1,loss_val,accu_val))
        if (i+1)%1000==0:
            test_data=CifarData(test_filenames,False) # 重新拿测试集
            all_test_accu_val=[]
            for j in range(test_steps):
                test_batch_data,test_batch_labels=test_data.next_batch(batch_size)
                test_accu_val=s.run([accuracy],
                                    feed_dict={x:test_batch_data,y:test_batch_labels})
                all_test_accu_val.append(test_accu_val)
            test_accu=np.mean(all_test_accu_val)
            print(f'now time={time.time()}')
            print('[Test ] Step: %d, accuracy: %4.5f' % (i+1,test_accu))
 

[Train] Step: 100, loss: 2.43925, accuracy: 0.15000
[Train] Step: 200, loss: 2.16907, accuracy: 0.20000
[Train] Step: 300, loss: 1.65611, accuracy: 0.35000
[Train] Step: 400, loss: 1.80444, accuracy: 0.35000
[Train] Step: 500, loss: 2.00781, accuracy: 0.20000
[Train] Step: 600, loss: 1.35989, accuracy: 0.60000
[Train] Step: 700, loss: 1.28732, accuracy: 0.55000
[Train] Step: 800, loss: 1.34693, accuracy: 0.55000
[Train] Step: 900, loss: 1.24280, accuracy: 0.45000
[Train] Step: 1000, loss: 1.16296, accuracy: 0.60000
(10000, 3072)
(10000,)
[3 8 8 0 6]
now time=1659018544.6309347
[Test ] Step: 1000, accuracy: 0.48850
[Train] Step: 1100, loss: 0.97391, accuracy: 0.75000
[Train] Step: 1200, loss: 0.95511, accuracy: 0.70000
[Train] Step: 1300, loss: 1.02998, accuracy: 0.65000
[Train] Step: 1400, loss: 1.43874, accuracy: 0.45000
[Train] Step: 1500, loss: 1.37702, accuracy: 0.40000
[Train] Step: 1600, loss: 0.96739, accuracy: 0.65000
[Train] Step: 1700, loss: 1.03788, accuracy: 0.80000
[Train]